# Initialization

In [1]:
import numpy as np
from pprint import pprint


from laboneq.contrib.example_helpers.generate_descriptor import generate_descriptor
#from laboneq.contrib.example_helpers.generate_device_setup import generate_device_setup
from laboneq.dsl.device import DeviceSetup




descriptor = generate_descriptor(
    #pqsc=[""], # 장비 여러개 사용시
    shfqc_6=["DEV12576"],
    number_data_qubits=5,
    multiplex=True,
    number_multiplex=5,
    include_cr_lines=True,
    include_ef_lines=True,
    get_zsync=False,  # Only set to True when using real device
    save = True,
    filename="BF7_KRISS",
    ip_address="10.172.11.217"
)


#descriptor
setup = DeviceSetup.from_descriptor(yaml_text=descriptor, server_host="10.172.11.217")
#setup
setup.instruments[0].device_options = 'SHFQC/PLUS/QC6CH'
#setup.instruments

In [2]:
from laboneq.simple import Session
session = Session(setup)
session.connect(ignore_version_mismatch=True, do_emulation=False)
#session.disconnect()

[2025.09.04 20:48:35.788] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /home/kriss/JSAHN/qubit-experiment/laboneq_output/log
[2025.09.04 20:48:35.789] INFO    VERSION: laboneq 2.57.0
[2025.09.04 20:48:35.790] INFO    Connecting to data server at 10.172.11.217:8004
[2025.09.04 20:48:35.795] INFO    Connected to Zurich Instruments LabOne Data Server version 25.07.0.507 at 10.172.11.217:8004
[2025.09.04 20:48:35.802] INFO    Configuring the device setup
[2025.09.04 20:48:35.809] INFO    The device setup is configured


In [3]:
from qpu_types.transmon import TransmonQubit, TransmonQubitParameters
from helper import load_qubit_parameters, save_qubit_parameters
from operations import TransmonOperations
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform



_loaded_qubits = load_qubit_parameters(filename="latest") 
#qubits = TransmonQubit.from_device_setup(setup) # Also can be done with load_qubit_parameters 
qubits = list(_loaded_qubits.values())

qubits = []
for i in range(5):
    q = TransmonQubit.from_logical_signal_group(
        f"q{i}",
        setup.logical_signal_groups[f"q{i}"],
        parameters=TransmonQubitParameters()
    )
    qubits.append(q)



qpu = QPU(quantum_elements=qubits, quantum_operations=TransmonOperations())
#quantum_platform = QuantumPlatform(setup=setup, qpu=qpu)

# qpu.topology.add_edge("bus", "q16", "q17")
# qpu.topology.add_edge("bus", "q17", "q18")
# qpu.topology.add_edge("bus", "q18", "q19")
# qpu.topology.plot()




from laboneq.simple import workflow
folder_store = workflow.logbook.FolderStore("./experiment_store") 
folder_store.activate()
#folder_store.deactivate()
workflow.logbook.LoggingStore().activate()
#workflow.logbook.LogbookStore().deactivate()


BF 7

In [4]:
qubits

[TransmonQubit(
│   uid='q0',
│   signals={
│   │   'drive': 'q0/drive',
│   │   'drive_ef': 'q0/drive_ef',
│   │   'drive_cr': 'q0/drive_cr',
│   │   'measure': 'q0/measure',
│   │   'acquire': 'q0/acquire'
│   },
│   parameters=TransmonQubitParameters(
│   │   custom={},
│   │   ge_T1=0,
│   │   ge_T2=0,
│   │   ge_T2_star=0,
│   │   ef_T1=0,
│   │   ef_T2=0,
│   │   ef_T2_star=0,
│   │   drive_lo_frequency=None,
│   │   readout_lo_frequency=None,
│   │   resonance_frequency_ge=None,
│   │   resonance_frequency_ef=None,
│   │   readout_resonator_frequency=None,
│   │   resonance_targ_frequency=None,
│   │   ge_drive_amplitude_pi=0.2,
│   │   ge_drive_amplitude_pi2=0.1,
│   │   ge_drive_length=5e-08,
│   │   ge_drive_pulse={
│   │   │   'function': 'drag',
│   │   │   'beta': 0,
│   │   │   'sigma': 0.25
│   │   },
│   │   ef_drive_amplitude_pi=0.2,
│   │   ef_drive_amplitude_pi2=0.1,
│   │   ef_drive_length=5e-08,
│   │   ef_drive_pulse={
│   │   │   'function': 'drag',
│   │   │   '

In [29]:
# 0 : 15 , 1 : 16 , 2 : 17, 3 : 18, 4 : 19
for q in qubits:
    #q.parameters.ge_drive_pulse["sigma"] = 0.25
    q.parameters.readout_amplitude = 1.0
    q.parameters.reset_delay_length = 20e-6

    q.parameters.readout_range_out = -25
    q.parameters.readout_range_in = 0
    q.parameters.drive_range = -10

    q.parameters.readout_lo_frequency = 7.0e9
    q.parameters.drive_lo_frequency = 4.8e9

    q.parameters.spectroscopy_length = 10e-6

    q.parameters.ge_drive_length =64e-9
    

 

qubits[2].parameters.resonance_frequency_ge = 4.716e9
qubits[3].parameters.resonance_frequency_ge = 4.664e9
qubits[4].parameters.resonance_frequency_ge = 4.779e9

qubits[2].parameters.readout_resonator_frequency = 6.9226e9
qubits[3].parameters.readout_resonator_frequency = 6.8433e9
qubits[4].parameters.readout_resonator_frequency = 6.7739e9



# 1. Spectroscopy

In [ ]:
from helper import adjust_amplitude_for_output_range, calculate_power
adjust_amplitude_for_output_range(initial_output_dbm=0, initial_amplitude= 0.002, new_output_dbm=-5)
calculate_power(output_dbm=0, amplitude=0.001,attenuation=[])

0.006324555320336759

### 1.1 Resonator spectroscopy with amplitude sweep

STRATEGY
1. CW spectroscpy for wide range scan
    * CW spectroscopy can be done for wide range scan 
    * amplitude, frequency 2D sweep 을 통해  power에 따른 broadening 및 SNR 경향 관찰
    * $\Delta F$ : frequency span,  $\Delta f_r$
    * Pick $\Delta F / \Delta f_r$ 1~2

In [ ]:
from experiments import resonator_spectroscopy_amplitude

####################################################################################
q=qubits[4]
pprint(q.readout_parameters())
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-10e6,10e6,501)
amplitudes = np.linspace(0.0, 1.0, 21)
#####################################################################################
options = resonator_spectroscopy_amplitude.experiment_workflow.options()
#workflow.show_fields(options)
options.update(True)
options.use_cw(False)
options.do_analysis(True)
######################################################################################
# Build and run resonator_spectroscopy_amplitude wormflow
res_spec_amp = resonator_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options
)
res_spec_amp_result=res_spec_amp.run()


### 1.2 Resonator Spectroscopy
* CW not recommended 

In [ ]:
from experiments import resonator_spectroscopy

####################################################################################
q=qubits[4]
q.parameters.readout_amplitude = 0.5
print(q.readout_parameters())
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#frequencies = q.parameters.readout_resonator_frequency + np.linspace(-10e6,10e6,501)
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-0.5e6,0.5e6,1001)
#####################################################################################
options = resonator_spectroscopy.experiment_workflow.options()
#workflow.show_fields(options)
options.count(1024*2)
options.update(False)
options.use_cw(False) #qop.measure 
options.fit_complex_resonator(False)
options.fit_lorentzian(True)
######################################################################################
# Build and run resonator_spectroscopy_amplitude wormflow
res_spec = resonator_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    options=options
)
res_spec_result = res_spec.run()
res_spec_analysis_result = res_spec_result.tasks["analysis_workflow"]
pprint(res_spec_analysis_result.output)

In [ ]:
res_spec_result.tasks["analysis_workflow"].tasks["fit_data"].output

In [ ]:
# res_spec_result.tasks["analysis_workflow"].tasks["plot_real_imaginary"].output

In [ ]:
from laboneq.simple import show_pulse_sheet
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

#show_pulse_sheet(compiled_experiment=res_spec_result.tasks["compile_experiment"].output,name='test', interactive=True)
#plot_simulation(compiled_experiment=res_spec_result.tasks["compile_experiment"].output)

### 2.1 Qubit Spectroscopy with Amplitude Sweep
* Current qubit spectroscopy modules do not support CW 
* Recommended to use long enough spectroscopy drive (pulsed) to see steady state signals

In [ ]:
qubits[2].parameters.drive_range = -5
qubits[2].parameters.drive

In [16]:
from experiments import qubit_spectroscopy_amplitude

###############################################################
q = qubits[4]
frequencies = q.parameters.resonance_frequency_ge + np.linspace(-2e6, 2e6, 101)
amplitudes = np.linspace(0.01, 0.002, 11)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")

pprint(q.readout_parameters())
pprint(q.spectroscopy_parameters())
################################################################
options = qubit_spectroscopy_amplitude.experiment_workflow.options()
options.update(False)
# workflow.show_fields(options)
###################################################################
qubit_spec_amp = qubit_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options
)

qubit_spec_amp_result = qubit_spec_amp.run()
qubit_spec_amp_result

DRIVE range (dBm) : 0, READOUT range (dBm) : -25
('measure',
 {'amplitude': 0.6,
  'length': 2e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
('drive',
 {'amplitude': 0.002,
  'length': 1e-05,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
[2025.09.04 21:02:58.832] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:02:58.833] INFO      Workflow 'qubit_spectroscopy_amplitude': execution started at 2025-09-04      
[2025.09.04 21:02:58.833] INFO      12:02:58.832495Z                                                              
[2025.09.04 21:02:58.833] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:02:58.846] INFO    Task 'temporary_qpu': started at 2025-09-04 12:02:58.845817Z
[2025.09.04 21:02:58.847] INFO    Task 'temporary_qpu': ended at 2025-09

WorkflowResult(name=qubit_spectroscopy_amplitude, index=())

### 2.2 Qubit Spectroscopy 


In [ ]:
from experiments import qubit_spectroscopy

q
###############################################################
q = qubits[4]
q.parameters.spectroscopy_amplitude = 0.002
q.parameters.readout_amplitude = 0.6
frequencies = q.parameters.resonance_frequency_ge + np.linspace(-3e6, 3e6, 501)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
pprint(q.readout_parameters())
pprint(q.spectroscopy_parameters())
################################################################
options = qubit_spectroscopy.experiment_workflow.options()
options.update(False)
# workflow.show_fields(options)
###################################################################
qubit_spec = qubit_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    options=options
)

qubit_spec_result = qubit_spec.run()
qubit_spec_result

DRIVE range (dBm) : -10, READOUT range (dBm) : -25
('measure',
 {'amplitude': 0.6,
  'length': 2e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
('drive',
 {'amplitude': 0.006,
  'length': 1e-05,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
[2025.09.04 21:22:09.772] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:22:09.772] INFO      Workflow 'qubit_spectroscopy': execution started at 2025-09-04                
[2025.09.04 21:22:09.772] INFO      12:22:09.772033Z                                                              
[2025.09.04 21:22:09.773] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:22:09.782] INFO    Task 'temporary_qpu': started at 2025-09-04 12:22:09.782505Z
[2025.09.04 21:22:09.784] INFO    Task 'temporary_qpu': ended at 2025-

WorkflowResult(name=qubit_spectroscopy, index=())

# 2. Rabi

## 2.1 Rabi Amplitude 

In [36]:
qubits[4].parameters.ge_drive_amplitude_pi = 0.2571
qubits[4].parameters.ge_drive_amplitude_pi2 = 0.1222

In [33]:
from laboneq_applications.experiments import amplitude_rabi
q = qubits[4]

#######################################################################
amplitudes = np.linspace(0.02, 1.0,501)
print(q.parameters.ge_drive_amplitude_pi)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#######################################################################
options = amplitude_rabi.experiment_workflow.options()
options.update(False)
options.use_cal_traces(False)
#print(workflow.show_fields(options))

###################################################################
rabi_amp = amplitude_rabi.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    amplitudes=amplitudes,
    options=options
)

rabi_amp_result = rabi_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(rabi_amp_result.tasks['analysis_workflow'].output)

0.2
DRIVE range (dBm) : -10, READOUT range (dBm) : -25
[2025.09.04 21:18:51.420] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:18:51.420] INFO      Workflow 'amplitude_rabi': execution started at 2025-09-04 12:18:51.419811Z   
[2025.09.04 21:18:51.421] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:18:51.431] INFO    Task 'temporary_qpu': started at 2025-09-04 12:18:51.431297Z
[2025.09.04 21:18:51.432] INFO    Task 'temporary_qpu': ended at 2025-09-04 12:18:51.432282Z
[2025.09.04 21:18:51.434] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-04 
[2025.09.04 21:18:51.435] INFO    12:18:51.434582Z
[2025.09.04 21:18:51.435] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-09-04 12:18:51.435721Z
[2025.09.04 21:18:51.437] INFO    Task 'create_experiment': started at 2025-09-04 12:18:51.436907Z
[2025.09.04 21:18:51.443] INFO    Task 'create

## 2.2 (optional) Rabi Amplitude Chevron 

In [20]:
from experiments import amplitude_rabi_chevron

#######################################################################
q = qubits[4]
amplitudes = np.linspace(0,0.5,11)
frequencies = q.parameters.resonance_frequency_ge + np.linspace(-3e6, 3e6, 51)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#######################################################################
options = amplitude_rabi_chevron.experiment_workflow.options()

#print(workflow.show_fields(options))

###################################################################
rabi_amp_chev = amplitude_rabi_chevron.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options
)

rabi_amp_chev_result = rabi_amp_chev.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(rabi_amp_chev_result.tasks['analysis_workflow'].output)


DRIVE range (dBm) : 0, READOUT range (dBm) : -25
[2025.09.04 21:11:27.995] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:11:27.995] INFO      Workflow 'amplitude_rabi_chevron': execution started at 2025-09-04            
[2025.09.04 21:11:27.996] INFO      12:11:27.994975Z                                                              
[2025.09.04 21:11:27.996] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:11:28.007] INFO    Task 'create_experiment': started at 2025-09-04 12:11:28.007238Z
[2025.09.04 21:11:28.014] INFO    Task 'create_experiment': ended at 2025-09-04 12:11:28.013843Z
[2025.09.04 21:11:28.015] INFO    Task 'compile_experiment': started at 2025-09-04 12:11:28.015275Z
[2025.09.04 21:11:28.017] INFO    Resolved modulation type of oscillator 'q4_readout_acquire_osc' on signal 'q4/acquire' to SOFTWARE
[2025.09.04 21:11:28.017] INFO    Resolved modulation type of 

## 2.3 (optional) Rabi Length
- Not working yet

In [ ]:
# from experiments import time_rabi
# #######################################################################
# q = qubits[2]
# q.parameters.ge_drive_amplitude_pi = 0.3
# lengths = np.linspace(0.0, 240e-9, 21 ),
# print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
# #######################################################################
# options = time_rabi.experiment_workflow.options()

# #print(workflow.show_fields(options))

# ###################################################################
# rabi_length = time_rabi.experiment_workflow(
#     session=session,
#     qpu=qpu,
#     qubits=q,
#     lengths=lengths,
#     options=options
# )

# rabi_length_result = rabi_length.run()
# #qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
# print(rabi_length_result.tasks['analysis_workflow'].output)


## 2.2 (optional) Rabi Length Chevron

In [ ]:
# from experiments import time_rabi_chevron
# #######################################################################
# q = qubits[4]

# frequencies = q.parameters.resonance_frequency_ge + np.linspace(-5e6, 5e6, 21)
# lengths = np.linspace(0.0, 240e-9, 21 ),
# print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
# #######################################################################
# options = time_rabi_chevron.experiment_workflow.options()

# #print(workflow.show_fields(options))

# ###################################################################
# rabi_length_chev = time_rabi_chevron.experiment_workflow(
#     session=session,
#     qpu=qpu,
#     qubits=q,
#     frequencies=frequencies,
#     lengths=lengths,
#     options=options
# )

# rabi_length_chev_result = rabi_length_chev.run()
# #qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
# print(rabi_length_chev_result.tasks['analysis_workflow'].output)


DRIVE range (dBm) : 0, READOUT range (dBm) : -25
[2025.09.04 21:11:10.036] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:11:10.036] INFO      Workflow 'time_rabi_chevron': execution started at 2025-09-04                 
[2025.09.04 21:11:10.037] INFO      12:11:10.035846Z                                                              
[2025.09.04 21:11:10.037] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:11:10.041] WARNING Type <class 'tuple'> not supported by the serializer [name: time_rabi_chevron.input.lengths].
[2025.09.04 21:11:10.047] INFO    Task 'create_experiment': started at 2025-09-04 12:11:10.047649Z
[2025.09.04 21:11:10.048] WARNING Type <class 'tuple'> not supported by the serializer [name: create_experiment.input.lengths].
[2025.09.04 21:11:10.053] ERROR   Task 'create_experiment': failed at 2025-09-04 12:11:10.052956Z with: 


[2025.09.04 21:11:10.053] ERROR   Task 'create_experiment': failed at 2025-09-04 12:11:10.052956Z with: 


[2025.09.04 21:11:10.053] ERROR   TypeError("'tuple' object does not support item assignment")


[2025.09.04 21:11:10.053] ERROR   TypeError("'tuple' object does not support item assignment")


[2025.09.04 21:11:10.054] INFO    Task 'create_experiment': ended at 2025-09-04 12:11:10.054008Z
[2025.09.04 21:11:10.055] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:11:10.055] INFO      Workflow 'time_rabi_chevron': execution ended at 2025-09-04 12:11:10.054957Z  
[2025.09.04 21:11:10.055] INFO     ────────────────────────────────────────────────────────────────────────────── 


TypeError: 'tuple' object does not support item assignment

# 3. Ramsey

In [43]:
from experiments import ramsey

q = qubits[4]
#######################################################################
delays = np.linspace(0,20e-6,301)
detunings = 0.4e6
pprint(q.readout_parameters())
#######################################################################
options = ramsey.experiment_workflow.options()
options.update(True)
options.count(1024)
options.use_cal_traces(True)
# Build and run Ramsey workflow (no explicit command table options required)
ramsey_wf = ramsey.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    delays=delays,
    detunings=detunings,
    options=options
)
ramsey_result = ramsey_wf.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(ramsey_result.tasks['analysis_workflow'].output)

('measure',
 {'amplitude': 0.6,
  'length': 2e-06,
  'pulse': {'function': 'GaussianSquare',
            'risefall_sigma_ratio': 3.0,
            'sigma': 0.2}})
[2025.09.04 21:32:57.789] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:32:57.790] INFO      Workflow 'ramsey': execution started at 2025-09-04 12:32:57.789681Z           
[2025.09.04 21:32:57.790] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:32:57.800] INFO    Task 'temporary_qpu': started at 2025-09-04 12:32:57.800013Z
[2025.09.04 21:32:57.801] INFO    Task 'temporary_qpu': ended at 2025-09-04 12:32:57.801033Z
[2025.09.04 21:32:57.803] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-04 
[2025.09.04 21:32:57.803] INFO    12:32:57.803162Z
[2025.09.04 21:32:57.804] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-09-04 12:32:57.804118Z
[2025.09.04 21:32:57.805] INFO    Task

In [48]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[4]
temporary_parameters = q.parameters.copy()

#######################################################################
repetitions =np.arange(1,50)
#######################################################################
options = amplitude_fine.experiment_workflow_x180.options()
workflow.show_fields(options)

Option Fields
acquisition_type:
	Description: Acquisition type to use for the experiment.
	Classes and Defaults: [('TuneupExperimentOptions', AcquisitionType.INTEGRATION)], 

active_reset:
	Description: Whether to use active reset.
	Classes and Defaults: [('TuneupExperimentOptions', False)], 

active_reset_repetitions:
	Description: The number of times to repeat the active resets.
	Classes and Defaults: [('TuneupExperimentOptions', 1)], 

active_reset_states:
	Description: The qubit states to actively reset.
	Classes and Defaults: [('TuneupExperimentOptions', 'ge')], 

averaging_mode:
	Description: Averaging mode to use for the experiment.
	Classes and Defaults: [('TuneupExperimentOptions', AveragingMode.CYCLIC)], 

cal_states:
	Description: The states to prepare in the calibration traces.
	Classes and Defaults: [('TuneupExperimentOptions', 'ge'), ('CalculateQubitPopulationOptions', 'ge'), ('PlotPopulationOptions', 'ge')], 

	Description: The states to prepare in the calibration traces

In [50]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[4]
temporary_parameters = q.parameters.copy()

#######################################################################
repetitions =np.arange(1,50)
#######################################################################
options = amplitude_fine.experiment_workflow_x180.options()
options.update(True)
options.use_cal_traces(False)
options.do_pca(False)

#print(workflow.show_fields(options))

###################################################################
error_amp = amplitude_fine.experiment_workflow_x180(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

error_amp_result = error_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_result.tasks['analysis_workflow'].output)

[2025.09.04 21:35:59.979] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:35:59.980] INFO      Workflow 'experiment_workflow_x180': execution started at 2025-09-04          
[2025.09.04 21:35:59.980] INFO      12:35:59.979321Z                                                              
[2025.09.04 21:35:59.980] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.04 21:35:59.982] WARNING Type <class 'dict'> not supported by the serializer [name: experiment_workflow_x180.input.temporary_parameters].
[2025.09.04 21:35:59.990] INFO    Task 'temporary_qpu': started at 2025-09-04 12:35:59.990541Z
[2025.09.04 21:35:59.991] WARNING Type <class 'dict'> not supported by the serializer [name: temporary_qpu.input.temporary_parameters].
[2025.09.04 21:35:59.992] INFO    Task 'temporary_qpu': ended at 2025-09-04 12:35:59.991927Z
[2025.09.04 21:35:59.994] INFO    Task 'temporary_quantum_elements_fr

# Basic Readout optimization
Here we optimize SNR via

* `readout_length`
* `readout-amplitude`

* `readout_integration_length`
* `readout_integration_delay`
* `readout_integration_weight`

In [ ]:
from experiments import dispersive_shift

In [ ]:
from experiments import time_traces

In [ ]:
from experiments import signal_propagation_delay

# Save Qubit Parameters

In [ ]:
from helper import load_qubit_parameters, save_qubit_parameters
save_qubit_parameters(qubits={ qubit.uid : qubit for qubit in qubits}, save_folder="./qubit_parameters", filename="KRISS")